In [1]:
import numpy as np 
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
df_sptn = pd.read_csv('Spaceship_train.csv')
df_sptn.head(5)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [3]:
df_sptn.Destination.value_counts(),df_sptn.HomePlanet.value_counts(),df_sptn.VIP.value_counts(),df_sptn.CryoSleep.value_counts()

(TRAPPIST-1e      5915
 55 Cancri e      1800
 PSO J318.5-22     796
 Name: Destination, dtype: int64,
 Earth     4602
 Europa    2131
 Mars      1759
 Name: HomePlanet, dtype: int64,
 False    8291
 True      199
 Name: VIP, dtype: int64,
 False    5439
 True     3037
 Name: CryoSleep, dtype: int64)

In [4]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,PowerTransformer,LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split,cross_val_score,cross_val_predict
from sklearn.metrics import accuracy_score

In [5]:
df_data = df_sptn.drop(columns = ['Name','Transported'])
lb = LabelEncoder()
df_target = lb.fit_transform(df_sptn['Transported'])

In [6]:
transf = ColumnTransformer(transformers=[
                                        ('tnf1',SimpleImputer(strategy='most_frequent'),['HomePlanet','CryoSleep','Cabin','Destination','VIP']),
                                        ('tnf2',SimpleImputer(strategy = 'median'),['Age','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck'])
                                        ],remainder='passthrough')
df_data1 = pd.DataFrame(transf.fit_transform(df_data),columns = ['HomePlanet','CryoSleep','Cabin','Destination','VIP','Age','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck','PassengerId'])


In [7]:
transf1 = ColumnTransformer(transformers = [('tnf3',OneHotEncoder(drop = 'first',),['HomePlanet','Destination','CryoSleep','VIP'])],
                                             remainder = 'passthrough')
df_data2 = pd.DataFrame(transf1.fit_transform(df_data1),columns = ['Europe','Mars','55 Cancri e','PSO J318.5-22','True','True','Cabin','Age','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck','PassengerId'])

In [8]:
df_data2.head()

,Europe,Mars,55 Cancri e,PSO J318.5-22,True,True,Cabin,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,PassengerId
0,1.0,0.0,0.0,1.0,0.0,0.0,B/0/P,39.0,0.0,0.0,0.0,0.0,0.0,0001_01
1,0.0,0.0,0.0,1.0,0.0,0.0,F/0/S,24.0,109.0,9.0,25.0,549.0,44.0,0002_01
2,1.0,0.0,0.0,1.0,0.0,1.0,A/0/S,58.0,43.0,3576.0,0.0,6715.0,49.0,0003_01
3,1.0,0.0,0.0,1.0,0.0,0.0,A/0/S,33.0,0.0,1283.0,371.0,3329.0,193.0,0003_02
4,0.0,0.0,0.0,1.0,0.0,0.0,F/1/S,16.0,303.0,70.0,151.0,565.0,2.0,0004_01


In [9]:
n1 = np.array(df_data2.PassengerId)
n2 = []
for i in range(len(n1)):
    n2.append(n1[i][:4]+n1[i][5:])
d2 = pd.DataFrame(n2,columns = ['PassengerId'])
# d1['PassengerId] = d1['PassengerId'].apply(pd.tp_numeric)
d2 = d2.astype(int)
d2

,PassengerId
0,101
1,201
2,301
3,302
4,401
...,...
8688,927601
8689,927801
8690,927901
8691,928001


In [10]:
df_data2['PassengerId'] = d2['PassengerId']
df_data2.drop(['Cabin'],axis = 1,inplace=True)
df_data2

,Europe,Mars,55 Cancri e,PSO J318.5-22,True,True,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,PassengerId
0,1.0,0.0,0.0,1.0,0.0,0.0,39.0,0.0,0.0,0.0,0.0,0.0,101
1,0.0,0.0,0.0,1.0,0.0,0.0,24.0,109.0,9.0,25.0,549.0,44.0,201
2,1.0,0.0,0.0,1.0,0.0,1.0,58.0,43.0,3576.0,0.0,6715.0,49.0,301
3,1.0,0.0,0.0,1.0,0.0,0.0,33.0,0.0,1283.0,371.0,3329.0,193.0,302
4,0.0,0.0,0.0,1.0,0.0,0.0,16.0,303.0,70.0,151.0,565.0,2.0,401
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,1.0,0.0,0.0,0.0,0.0,1.0,41.0,0.0,6819.0,0.0,1643.0,74.0,927601
8689,0.0,0.0,1.0,0.0,1.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,927801
8690,0.0,0.0,0.0,1.0,0.0,0.0,26.0,0.0,0.0,1872.0,1.0,0.0,927901
8691,1.0,0.0,0.0,0.0,0.0,0.0,32.0,0.0,1049.0,0.0,353.0,3235.0,928001


In [11]:
transf2 = ColumnTransformer(transformers=[('tnf4',PowerTransformer(),['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck'])],remainder = 'passthrough')
df_data3 = pd.DataFrame(transf2.fit_transform(df_data2),columns = ['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck','Europe','Mars','55 Cancri e','PSO J318.5-22','True','True','Age','PassengerId'])
df_data3

,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Europe,Mars,55 Cancri e,PSO J318.5-22,True,True,Age,PassengerId
0,-0.689373,-0.707127,-0.681519,-0.726569,-0.69971,1.0,0.0,0.0,1.0,0.0,0.0,39.0,101
1,1.453068,0.769003,1.27459,1.55844,1.290862,0.0,0.0,0.0,1.0,0.0,0.0,24.0,201
2,1.302856,1.655162,-0.681519,1.681922,1.311791,1.0,0.0,0.0,1.0,0.0,1.0,58.0,301
3,-0.689373,1.60778,1.633217,1.658768,1.510333,1.0,0.0,0.0,1.0,0.0,0.0,33.0,302
4,1.56119,1.300712,1.559902,1.560675,0.236828,0.0,0.0,0.0,1.0,0.0,0.0,16.0,401
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,-0.689373,1.676461,-0.681519,1.627819,1.383911,1.0,0.0,0.0,0.0,0.0,1.0,41.0,927601
8689,-0.689373,-0.707127,-0.681519,-0.726569,-0.69971,0.0,0.0,1.0,0.0,1.0,0.0,18.0,927801
8690,-0.689373,-0.707127,1.705813,-0.121561,-0.69971,0.0,0.0,0.0,1.0,0.0,0.0,26.0,927901
8691,-0.689373,1.595977,-0.681519,1.520639,1.684045,1.0,0.0,0.0,0.0,0.0,0.0,32.0,928001


In [12]:
# x_train,x_test,y_train,y_test = train_test_split(df_data3,df_target,train_size=1)

In [13]:
lr = LogisticRegression()
lr.fit(df_data3,df_target)


LogisticRegression()

In [14]:
dt = DecisionTreeClassifier(criterion = 'entropy',max_depth = 8)
dt.fit(df_data3,df_target)


DecisionTreeClassifier(criterion='entropy', max_depth=8)

In [15]:
# yhat = dt.predict(x_test)
np.mean(cross_val_score(dt,df_data3,df_target,cv = 9)),np.mean(cross_val_score(lr,df_data3,df_target,cv = 6))



(0.7453105053690772, 0.5036235855807077)

In [16]:
df_test_data = pd.read_csv('Spaceship_test.csv')
df_test_data.drop(columns = ['Name'],inplace=True)

In [17]:
transf3 = ColumnTransformer(transformers=[
                                        ('tnf1',SimpleImputer(strategy='most_frequent'),['HomePlanet','CryoSleep','Cabin','Destination','VIP']),
                                        ('tnf2',SimpleImputer(strategy = 'median'),['Age','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck'])
                                        ],remainder='passthrough')
df_data4 = pd.DataFrame(transf3.fit_transform(df_test_data),columns = ['HomePlanet','CryoSleep','Cabin','Destination','VIP','Age','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck','PassengerId'])


In [18]:
transf4 = ColumnTransformer(transformers = [('tnf3',OneHotEncoder(drop = 'first',),['HomePlanet','Destination','CryoSleep','VIP'])],
                                             remainder = 'passthrough')
df_data5= pd.DataFrame(transf4.fit_transform(df_data4),columns = ['Europe','Mars','55 Cancri e','PSO J318.5-22','True','True','Cabin','Age','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck','PassengerId'])

In [19]:
n3 = np.array(df_data5.PassengerId)
n4 = []
for i in range(len(n3)):
    n4.append(n3[i][:4]+n3[i][5:])
d9 = pd.DataFrame(n3,columns = ['PassengerId'])
# d1['PassengerId] = d1['PassengerId'].apply(pd.tp_numeric)
d9 = d9.astype(int)
df_data5['PassengerId'] = d9['PassengerId']
df_data5.drop(['Cabin'],axis = 1,inplace=True)
df_data5


,Europe,Mars,55 Cancri e,PSO J318.5-22,True,True,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,PassengerId
0,0.0,0.0,0.0,1.0,1.0,0.0,27.0,0.0,0.0,0.0,0.0,0.0,1301
1,0.0,0.0,0.0,1.0,0.0,0.0,19.0,0.0,9.0,0.0,2823.0,0.0,1801
2,1.0,0.0,0.0,0.0,1.0,0.0,31.0,0.0,0.0,0.0,0.0,0.0,1901
3,1.0,0.0,0.0,1.0,0.0,0.0,38.0,0.0,6652.0,0.0,181.0,585.0,2101
4,0.0,0.0,0.0,1.0,0.0,0.0,20.0,10.0,0.0,635.0,0.0,0.0,2301
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,0.0,0.0,0.0,1.0,1.0,0.0,34.0,0.0,0.0,0.0,0.0,0.0,926602
4273,0.0,0.0,0.0,1.0,0.0,0.0,42.0,0.0,847.0,17.0,10.0,144.0,926901
4274,0.0,1.0,0.0,0.0,1.0,0.0,26.0,0.0,0.0,0.0,0.0,0.0,927101
4275,1.0,0.0,0.0,1.0,0.0,0.0,26.0,0.0,2680.0,0.0,0.0,523.0,927301


In [20]:
transf5 = ColumnTransformer(transformers=[('tnf4',PowerTransformer(),['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck'])],remainder = 'passthrough')
df_data6 = pd.DataFrame(transf5.fit_transform(df_data5),columns = ['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck','Europe','Mars','55 Cancri e','PSO J318.5-22','True','True','Age','PassengerId'])
df_data6

,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Europe,Mars,55 Cancri e,PSO J318.5-22,True,True,Age,PassengerId
0,-0.696518,-0.700731,-0.685448,-0.726096,-0.685248,0.0,0.0,0.0,1.0,1.0,0.0,27.0,1301
1,-0.696518,0.784637,-0.685448,1.658785,-0.685248,0.0,0.0,0.0,1.0,0.0,0.0,19.0,1801
2,-0.696518,-0.700731,-0.685448,-0.726096,-0.685248,1.0,0.0,0.0,0.0,1.0,0.0,31.0,1901
3,-0.696518,1.678074,-0.685448,1.459015,1.636465,1.0,0.0,0.0,1.0,0.0,0.0,38.0,2101
4,0.894415,-0.700731,1.650548,-0.726096,-0.685248,0.0,0.0,0.0,1.0,0.0,0.0,20.0,2301
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,-0.696518,-0.700731,-0.685448,-0.726096,-0.685248,0.0,0.0,0.0,1.0,1.0,0.0,34.0,926602
4273,-0.696518,1.58801,1.152298,0.823102,1.519358,0.0,0.0,0.0,1.0,0.0,0.0,42.0,926901
4274,-0.696518,-0.700731,-0.685448,-0.726096,-0.685248,0.0,1.0,0.0,0.0,1.0,0.0,26.0,927101
4275,-0.696518,1.647328,-0.685448,-0.726096,1.629613,1.0,0.0,0.0,1.0,0.0,0.0,26.0,927301


In [21]:
yhat1 = dt.predict(df_data6)
yhat2 = lr.predict(df_data6)

In [22]:
df_submission_dt = df_test_data[['PassengerId']]
df_submission_dt['Transported']=pd.DataFrame(yhat1).replace([0,1],['False','True'])

df_submission_lr = df_test_data[['PassengerId']]
df_submission_lr['Transported']=pd.DataFrame(yhat2).replace([0,1],['False','True'])

C:\Users\Ayush Jha\AppData\Local\Temp\ipykernel_15680\3870502738.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_submission_dt['Transported']=pd.DataFrame(yhat1).replace([0,1],['False','True'])
C:\Users\Ayush Jha\AppData\Local\Temp\ipykernel_15680\3870502738.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_submission_lr['Transported']=pd.DataFrame(yhat2).replace([0,1],['False','True'])


In [23]:

df_submission_dt.to_csv('submission_dt.csv',index = False)
df_submission_lr.to_csv('submission_lr.csv',index = False)

In [24]:
df_submission_dt

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
...,...,...
4272,9266_02,True
4273,9269_01,False
4274,9271_01,True
4275,9273_01,False
